In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import re
from bs4 import BeautifulSoup
import numpy as np
import torch
import torch.nn as nn

In [ ]:
MODEL_PATH = "/kaggle/input/toxic-model-files/model_files/model_files_fold/model"
TOKENIZER_PATH = "/kaggle/input/toxic-model-files/model_files/model_files_fold/tokenizer"
MAX_LEN = 512
BATCH_SIZE = 16
DEVICE = 'cuda'
CLASSIFY_MODEL_PATH = '/kaggle/input/toxic-model-files/model_files/model_files_fold/best_model_classify'
RUDDIT_MODEL_PATH = '/kaggle/input/toxic-model-files/model_files/model_files_fold/best_model_ruddit'
RANK_MODEL_PATH = '/kaggle/input/toxic-model-files/final_model_output_2'
TEST_FILE_PATH = '/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv'
FOLDS = 5

device = torch.device(DEVICE)

# Data Reader

In [ ]:
class BatchEnabler:
    def __init__(self, data):
        self.tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)
        self.text_encodings = self.tokenizer(data['text'].to_list(), truncation=True, padding='max_length', max_length=MAX_LEN)
        self.comment_id_list = data['comment_id'].to_list()
    def __getitem__(self, idx):
        return {
            'input_ids' : torch.tensor(self.text_encodings.input_ids[idx], dtype=torch.long),
            'attention_mask' : torch.tensor(self.text_encodings.attention_mask[idx], dtype=torch.long),
            'comment_id' : self.comment_id_list[idx],
        }
    def __len__(self):
        return len(self.comment_id_list)

df_file = pd.read_csv(TEST_FILE_PATH)
batch_enabled = BatchEnabler(df_file)
data_loader = DataLoader(batch_enabled, 
                        batch_size = BATCH_SIZE)


# Define The Model Architecture

In [ ]:
class ToxicScoreModel(nn.Module):
    def __init__(self):
        super(ToxicScoreModel, self).__init__()
        self.bert_layer = AutoModel.from_pretrained(MODEL_PATH)
        self.bert_drop_1 = nn.Dropout(0.2)
        self.bert_drop_2 = nn.Dropout(0.2)
        self.l0_mean = nn.Linear(768, 128)
        self.l0_max = nn.Linear(768, 128)
        self.l1 = nn.Linear(256, 1)
    def forward(self, ids, mask):
        bert_out_text = self.bert_layer(
            input_ids = ids,
            attention_mask = mask
            )
        output = self.bert_drop_1(bert_out_text['last_hidden_state'].mean(axis=1))
        output_mean = self.l0_mean(output)
        output = self.bert_drop_1(bert_out_text['last_hidden_state'].max(axis=1)[0])
        output_max = self.l0_max(output)
        output = self.bert_drop_2(torch.cat((output_max, output_mean), 1))
        output = self.l1(output)
        return output

# RUDDIT MODEL SCORE INFERENCE

In [ ]:
final_toxic_ruddit = np.zeros((len(df_file), FOLDS))

for fold in range(FOLDS):
    model = ToxicScoreModel()
    model.load_state_dict(torch.load(f"{RUDDIT_MODEL_PATH}_{fold}"))
    model.to(device)
    model.eval()

    toxic_ruddit = []
    for _, dataset in tqdm(enumerate(data_loader), total=len(data_loader)): 
        input_ids = dataset['input_ids'].to(device, dtype = torch.long)
        attention_mask = dataset['attention_mask'].to(device, dtype = torch.long)
        with torch.no_grad():
            model_output = model(
                input_ids, 
                attention_mask, 
            )
        toxic_ruddit.extend(model_output.view(-1).detach().cpu().numpy().tolist())

    del model
    final_toxic_ruddit[:, fold] = np.array(toxic_ruddit)


# CLASSIFY MODEL SCORE INFERENCE

In [ ]:
final_toxic_classify = np.zeros((len(df_file), FOLDS))

for fold in range(FOLDS):
    model = ToxicScoreModel()
    model.load_state_dict(torch.load(f"{CLASSIFY_MODEL_PATH}_{fold}"))
    model.to(device)
    model.eval()

    toxic_classify = []
    for _, dataset in tqdm(enumerate(data_loader), total=len(data_loader)): 
        input_ids = dataset['input_ids'].to(device, dtype = torch.long)
        attention_mask = dataset['attention_mask'].to(device, dtype = torch.long)
        with torch.no_grad():
            model_output = model(
                input_ids, 
                attention_mask, 
            )
        toxic_classify.extend(model_output.view(-1).detach().cpu().numpy().tolist())

    del model
    final_toxic_classify[:, fold] = np.array(toxic_classify)

# FINAL ENSEMBLE LAYER

In [ ]:
final_toxic_classify = final_toxic_classify.mean(axis=1)
final_toxic_ruddit = final_toxic_ruddit.mean(axis=1)

ranking_input = np.concatenate((final_toxic_classify.reshape(-1,1), final_toxic_ruddit.reshape(-1,1)), axis = 1)

class BinaryClassification(nn.Module):
    def __init__(self, hidden_input):
        super(BinaryClassification, self).__init__()
        self.layer_1 = nn.Linear(hidden_input, 1)
        self.relu = nn.ReLU()
    def forward(self, inputs):
        x = self.relu(inputs)
        x = self.layer_1(x)
        return x

model = BinaryClassification(2)
model.load_state_dict(torch.load(RANK_MODEL_PATH))
model.eval()

final_output = model(torch.tensor(ranking_input, dtype=torch.float)).view(-1).detach().numpy().tolist()

# Create Submission File

In [ ]:
submission = pd.DataFrame()

submission['comment_id'] = df_file['comment_id']
submission['score'] = final_output

submission.to_csv('submission.csv', index=False)